## Exploratory Data Analysis

*Coding along with third edition of the online version of __[Think Stats](https://allendowney.github.io/ThinkStats/chap01.html)__ by Allen Downey.*

#### __Tools of statistics that will be used:__

1. __Data collection__: We will use data from a large national survey that was designed explicitly with the goal of generating statistically valid inferences about the U.S. population.

2. __Descriptive statistics__: We will generate statistics that summarize the data concisely, and evaluate different ways to visualize data.

3. __Exploratory data analysis__: We will look for patterns, differences, and other features that address the questions we are interested in. At the same time we will check for inconsistencies and identify limitations.

4. __Estimation__: We will use data from a sample to estimate characteristics of the general population.

4. __Hypothesis testing__: Where we see apparent effects, like a difference between two groups, we will evaluate whether the effect might have happened by chance.

(Source: https://allendowney.github.io/ThinkStats/chap01.html)

__Data collection__: The data used will be from surveys conducted by the __U.S. Centers for Disease Control and Prevention (CDC)__. Since 1973 their __National Survey of Family Growth (NSFG)__ is intended to gather “information on family life, marriage and divorce, pregnancy, infertility, use of contraception, and men’s and women’s health. 

The question we want to answer with the survey data that we'tre using is whether first babies tend to come late among some other questions.

In [1]:
from statadict import parse_stata_dict
import pandas as pd
import numpy as np

#### __The `statadict` Python Package__

The `statadict` package is part of the book's supporting materials rather than a general-purpose Python package on PyPI. It can be found in the book's GitHub repository along with other code examples and datasets. It got installed with `poetry add statadict`.

The `statadict` package in "Think Stats" is specifically designed to work with the National Survey of Family Growth (NSFG) dataset, which is a key dataset used throughout the book for teaching statistical concepts.

Usage of `statadict` in the book:

- Used primarily in the early chapters to load pregnancy data
- Helps analyze demographic and health statistics
- Forms the foundation for many statistical examples in the book

Working with NSFG Data:

- The NSFG is a national survey conducted by the CDC
- Contains data about family life, marriage, divorce, pregnancy
- The data files are in fixed-width format with accompanying .dct files
- The package helps convert this into usable Python data structures

#### __The .dct File Format__

A .dct (dictionary) file is a format used by Stata to describe the structure of fixed-width data files. Here's why they're used and their advantages:

1. Dictionary (.dct) File Structure:
```stata
dictionary using "data.raw" {
    str20  name     %20s  "Person's name"
    int    age      %2f   "Age in years"
    float  income   %8.2f "Annual income"
    *      _column(31)
}
```
This tells us:
- Variable names (name, age, income)
- Data types (str20, int, float)
- Column widths (%20s, %2f, %8.2f)
- Variable descriptions (in quotes)
- Column positions (_column)

2. Advantages over CSV:
   - Works with fixed-width files where data fields have precise character positions
   - Handles legacy data formats that predate CSV
   - Maintains exact field widths which can be crucial for certain data types
   - Better handling of missing values and special codes
   - Includes metadata like variable descriptions
   - More precise control over data types

3. Fixed-width vs CSV:
```
# Fixed-width format:
John Smith           45  50000.00
Jane Doe             32  65000.00

# CSV format:
name,age,income
"John Smith",45,50000.00
"Jane Doe",32,65000.00
```

The fixed-width format was common in older systems and is still used by some government agencies and research institutions, particularly for large datasets like census data or survey results. ***The NSFG data used in "Think Stats" uses this format, which is why the `statadict` package is needed to parse it.***

In [2]:
dct_file = "../assets/data/2002FemPreg.dct"
dat_file = "../assets/data/2002FemPreg.dat.gz"

__The .dct file (2002FemPreg.dct):__
- Contains the metadata/structure definition
- Tells Python how to interpret the data file
- Specifies column names, positions, and widths
- Acts like a "map" or "schema" for reading the data


__The .dat.gz file (2002FemPreg.dat.gz):__
- Contains the actual data in fixed-width format
- Compressed using gzip (.gz extension)
- Raw data without any column headers or structure information
- Just rows of data where each piece of information takes up a specific number of characters

In [19]:
# function takes meta data and actual data file names as arguments, 
# reads the dictionary (.dct),
# uses the results to read the data file
def read_stata(dct_file, dat_file):
    stata_dict = parse_stata_dict(dct_file)
    # read file with pandas' read_fwf
    resp = pd.read_fwf(
        dat_file,
        names=stata_dict.names,
        colspecs=stata_dict.colspecs,
        compression="gzip",
    )
    return resp

#### __The `pd.read_fwf()` function in pandas__

The `pd.read_fwf()` function in pandas is specifically designed to read Fixed-Width Formatted (FWF) data files. Let's break down what it's doing in this context:

1. What FWF means:
```
# Example of fixed-width format:
John Smith    35   Chicago    
Mary Johnson  42   New York   
Tom Brown     28   Boston     
```
Each field takes up a fixed number of characters, including spaces. For example:
- Name: 12 characters
- Age: 4 characters
- City: 11 characters

2. In the code example:
```python
resp = pd.read_fwf(
    dat_file,                # The data file to read
    names=stata_dict.names,  # Column names from the .dct file
    colspecs=stata_dict.colspecs,  # Where each column starts and ends
    compression="gzip"       # File is compressed
)
```

The `colspecs` parameter is crucial - it's a list of tuples that specify the exact character positions where each column starts and ends. For example:
```python
# Example colspecs might look like:
colspecs = [
    (0, 12),    # First column: characters 0-11
    (12, 16),   # Second column: characters 12-15
    (16, 27)    # Third column: characters 16-26
]
```

The function reads the fixed-width file line by line, splitting each line into columns based on these character positions, and creates a pandas DataFrame with the specified column names.

In [20]:
preg = read_stata(dct_file, dat_file)

In [21]:
# attribute shape contains the number of rows and columns
preg.shape

(13593, 243)

In [22]:
# method head displays the first few rows
# first column is the index of the DataFrame
preg.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,poverty_i,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,3410.389399,3869.349602,6448.271112,2,9,1231
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,3410.389399,3869.349602,6448.271112,2,9,1231
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,0,0,0,7226.301740,8567.549110,12999.542264,2,12,1231
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,7226.301740,8567.549110,12999.542264,2,12,1231
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,7226.301740,8567.549110,12999.542264,2,12,1231


In [23]:
# attribute columns contains the names of the columns
# column names are contained in an Index object (pandas data structure)
preg.columns

Index(['caseid', 'pregordr', 'howpreg_n', 'howpreg_p', 'moscurrp', 'nowprgdk',
       'pregend1', 'pregend2', 'nbrnaliv', 'multbrth',
       ...
       'poverty_i', 'laborfor_i', 'religion_i', 'metro_i', 'basewgt',
       'adj_mod_basewgt', 'finalwgt', 'secu_p', 'sest', 'cmintvw'],
      dtype='object', length=243)

In [24]:
# accessing a column from a DataFrame with the column name as a key
pregordr = preg["pregordr"]
type(pregordr) # result isPandas Series which represents a sequence of values

pandas.core.series.Series

In [25]:
pregordr.head() # Series also provides head

0    1
1    2
2    1
3    2
4    3
Name: pregordr, dtype: int64

#### __Variables of The NSFG dataset we’ll use for the exploration:__

- __caseid__ is the integer ID of the respondent

- __pregordr__ is a pregnancy serial number; for example, the code for a respondent’s first pregnancy is 1, for the second pregnancy is 2, and so on

- __prglngth__ is the integer duration of the pregnancy in weeks

- __outcome__ is an integer code for the outcome of the pregnancy. The code 1 indicates a live birth

- __birthord__ is a serial number for live births; the code for a respondent’s first child is 1, and so on; for outcomes other than live birth, this field is blank

- __birthwgt_lb__ and __birthwgt_oz__ contain the pounds and ounces parts of the birth weight of the baby

- __agepreg__ is the mother’s age at the end of the pregnancy

- __finalwgt__ is the statistical weight associated with the respondent; it is a floating-point value that indicates the number of people in the U.S. population this respondent represents

Many of the variables are __recodes__ - they are not part of the raw data collected by the survey but have been calculated using the raw data.

### Validation of the Data

One way to validate data is to compute basic statistics and compare them with published results. The NSFG codebook includes the followimng table which encodes the outcome of each pregnancy. The “Total” column indicates the number of pregnancies with each outcome.

<table border="1" class="dataframe" align="left">
  <thead>
    <tr style="text-align: right;">
      <th>Value</th>
      <th>Label</th>
      <th>Total</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>LIVE BIRTH</td>
      <td>9148</td>
    </tr>
    <tr>
      <td>2</td>
      <td>INDUCED ABORTION</td>
      <td>1862</td>
    </tr>
    <tr>
      <td>3</td>
      <td>STILLBIRTH</td>
      <td>120</td>
    </tr>
    <tr>
      <td>4</td>
      <td>MISCARRIAGE</td>
      <td>1921</td>
    </tr>
    <tr>
      <td>5</td>
      <td>ECTOPIC PREGNANCY</td>
      <td>190</td>
    </tr>
    <tr>
      <td>6</td>
      <td>CURRENT PREGNANCY</td>
      <td>352</td>
    </tr>
    <tr>
      <td>Total</td>
      <td></td>
      <td>13593</td>
    </tr>
  </tbody>
</table>

In [26]:
# checking the totals with the value_counts method
# it counts the number of times each value appears
# sort_index, which sorts the results according to the values in the Index (the left column)
preg["outcome"].value_counts().sort_index()

outcome
1    9148
2    1862
3     120
4    1921
5     190
6     352
Name: count, dtype: int64

Published table for `birthwgt_lb` (only recorded for pregnancies that ended in a live birth):

<table border="1" class="dataframe" align="left">
  <thead>
    <tr style="text-align: right;">
      <th>Value</th>
      <th>Label</th>
      <th>Total</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>.</td>
      <td>inapplicable</td>
      <td>4449</td>
    </tr>
    <tr>
      <td>0-5</td>
      <td>UNDER 6 POUNDS</td>
      <td>1125</td>
    </tr>
    <tr>
      <td>6</td>
      <td>6 POUNDS</td>
      <td>2223</td>
    </tr>
    <tr>
      <td>7</td>
      <td>7 POUNDS</td>
      <td>3049</td>
    </tr>
    <tr>
      <td>8</td>
      <td>8 POUNDS</td>
      <td>1889</td>
    </tr>
    <tr>
      <td>9-95</td>
      <td>9 POUNDS OR MORE</td>
      <td>799</td>
    </tr>
    <tr>
      <td>97</td>
      <td>Not ascertained</td>
      <td>1</td>
    </tr>
    <tr>
      <td>98</td>
      <td>REFUSED</td>
      <td>1</td>
    </tr>
    <tr>
      <td>99</td>
      <td>DON'T KNOW</td>
      <td>57</td>
    </tr>
    <tr>
      <td>Total</td>
      <td></td>
      <td>13593</td>
    </tr>
  </tbody>
</table>

In [27]:
# same as above, this time with dropna=False
counts = preg["birthwgt_lb"].value_counts(dropna=False).sort_index()
counts

birthwgt_lb
0.0        8
1.0       40
2.0       53
3.0       98
4.0      229
5.0      697
6.0     2223
7.0     3049
8.0     1889
9.0      623
10.0     132
11.0      26
12.0      10
13.0       3
14.0       3
15.0       1
51.0       1
97.0       1
98.0       1
99.0      57
NaN     4449
Name: count, dtype: int64

In [28]:
# check the counts for the weight range from 0 to 5 pounds
# using the loc attribute called loc
# loc is short for “location” (slices index to select a subset of the counts)
counts.loc[0:5]

birthwgt_lb
0.0      8
1.0     40
2.0     53
3.0     98
4.0    229
5.0    697
Name: count, dtype: int64

In [29]:
# the sum over the slice
counts.loc[0:5].sum() # same as in table above

np.int64(1125)

#### __Handling missing data__
We have three cases where the birth weight is unknown: the values 97, 98, and 99. Plus once case where the value is clearly wrong: 51 pounds. The way to handle missing data we will use here is replacing these values with the NaN value from NumPy.

In [ ]:
preg["birthwgt_lb"] = preg["birthwgt_lb"].replace([51, 97, 98, 99], np.nan)

### Transformation

Transformation is a kind of data cleaning where we have to convert data into different formats to perform some kind of calculation with them.

The variable `agepreg` contains the mother’s age at the end of the pregnancy. According to the codebook it is an integer number of centiyears (hundredths of a year).

In [ ]:
preg["agepreg"].mean()

In [ ]:
# converting the mean to years by dividing the variable through 100
preg["agepreg"] /= 100
preg["agepreg"].mean()

The columns `birthwgt_lb` and `birthwgt_oz` contain birth weights with the pounds and ounces in separate columns. We'll combine them into as single column that contains weights in pounds and fractions of a pound.

In [ ]:
# cleaning birthwgt_oz as we did with birthwgt_lb
preg["birthwgt_oz"].value_counts(dropna=False).sort_index()

In [ ]:
preg["birthwgt_oz"] = preg["birthwgt_oz"].replace([97, 98, 99], np.nan)

In [ ]:
# using the cleaned values to create a new column that combines pounds and ounces into a single quantity
preg["totalwgt_lb"] = preg["birthwgt_lb"] + preg["birthwgt_oz"] / 16.0
preg["totalwgt_lb"].mean() # average of the result seems plausible

### Summary Statistics

In [ ]:
# a Series object has a count method that returns the number of values that are not nan
weights = preg["totalwgt_lb"]
n = weights.count()
n

In [ ]:
# sum method that returns the sum of the values
weights.sum()

In [ ]:
# use the sum method to compute the mean
mean = weights.sum() / n
mean

In [ ]:
weights.mean() # same with the mean method

In [ ]:
# variance is a statistic that quantifies the spread of a set of values
# it is the mean of the squared deviations (which are the distances of each point from the mean)
# squared deviations:
squared_deviations = (weights - mean) ** 2
squared_deviations

In [ ]:
# mean of the squared deviations -> variance
var = squared_deviations.sum() / n

In [ ]:
weights.var() # var method of Series does the same thing

The result of `weights.var()` is slightly different because when the var method computes the mean of the squared deviations, it divides by `n-1` rather than `n`. To get the variance with `n` in the denominator you can get it by passing `ddof=0` as a keyword argument to the var method.

In [ ]:
weights.var(ddof=0)

So far we have variance of the birth weights (about 1.98) which is in units of pounds squared and therefore hard to interpret. A better option to describe a dataset is the __standard deviation__ which is the square root of variance. Informally values that are one or two standard deviations from the mean are common. On the other hand values farther from the mean are rare.

In [ ]:
std = np.sqrt(var)
std

In [ ]:
weights.std(ddof=0) # same with the std method

### Interpretation

To work with data effectively you have to think on the *level of statistics* and the *level of context* at the same time. Let's have a look at a subset of the data table.

In [ ]:
# selecting the rows in the pregnancy file with caseid 10229
# the query method takes a string that can contain column names, comparison operators, and numbers, among other things
subset = preg.query("caseid == 10229")
subset.shape # the respondent reports seven pregnancies

In [ ]:
subset["outcome"].values # outcomes, which are recorded in chronological order
# outcome code 1 indicates a live birth, code 4 indicates a miscarriage

### Glossary

__Anecdotal evidence__: Data collected informally from a small number of individual cases, often without systematic sampling.

__Cross-sectional study__: A study that collects data from a representative sample of a population at a single point or interval in time.

__Longitudinal study__: A study that collects data from the same group of respondents at more than one interval in time.

__Cycle__: One data-collection interval in a study that collects data at multiple intervals in time.

__Population__: The entire group of individuals or items that is the subject of a study.

__Sample__: A subset of a population, often chosen at random.

__Respondents__: People who participate in a survey and respond to questions.

__Representative__: A sample is representative if it is similar to the population in ways that are important for the purposes of the study.

__Stratified__: A sample is stratified if it deliberately oversamples some groups, usually to make sure that enough members are included to support valid conclusions.

__Oversampled__: A subset of the population is oversampled if its members have a higher chance of appearing in a sample.

__Codebook__: A document that describes the variables in a dataset, and provides other information about the data.

__Recode__: A variable that is computed based on other variables in a dataset.

__Raw data__: Data that has not been processed after collection.

__Data cleaning__: A process for identifying and correcting errors in a dataset, dealing with missing values, and computing recodes.

__Statistic__: A value that describes or summarizes a property of a sample.

__Standard deviation__: A statistic that quantifies the spread of data around the mean.

*(Source: https://allendowney.github.io/ThinkStats/chap01.html#glossary)*

### Exercises

#### __Exercise # 1__
Select the `birthord` column from preg, print the value counts, and compare to results published in the codebook at https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Dataset_Documentation/NSFG/Cycle6Codebook-Pregnancy.pdf.

<pre>
BIRTHORD FROM CODEBOOK (278 - 279)
Birth order (recode)
Value Label Total
. INAPPLICABLE 4445
01 1ST BIRTH 4413
02 2ND BIRTH 2874
03 3RD BIRTH 1234
04 4TH BIRTH 421
05 5TH BIRTH 126
06 6TH BIRTH 50
07 7TH BIRTH 20
08 8TH BIRTH 7
09 9TH BIRTH 2
10 10TH BIRTH 1
Total 13593
</pre>

In [ ]:
birthord = preg["birthord"].value_counts(dropna=False).sort_index()
birthord # same as in codebook

#### __Exercise # 2__

Create a new column named `totalwgt_kg` that contains birth weight in kilograms (there are approximately 2.2 pounds per kilogram). Compute the mean and standard deviation of the new column.

In [ ]:
# above we've created the column totalwgt_lb with the total weight in pounds
preg["totalwgt_lb"]

In [ ]:
# new column totalwgt_kg that contains birth weight in kilograms
# there are approximately 2.2 pounds per kilogram
preg["totalwgt_kg"] = preg["totalwgt_lb"] / 2.205
preg["totalwgt_kg"]

In [ ]:
# Computing the mean of the new column
preg["totalwgt_kg"].mean()

In [ ]:
# computing the standard deviation
preg["totalwgt_kg"].std(ddof=0)

#### __Exercise # 3__

What are the pregnancy lengths for the respondent with caseid 2298?

What was the birth weight of the first baby born to the respondent with caseid 5013? Hint: You can use and to check more than one condition in a query.

In [ ]:
preg.head()

In [ ]:
subset_2298 = preg.query("caseid == 2298")
subset_2298

In [ ]:
# wksgest (46 - 47): Gestational length of completed pregnancy (in weeks) (computed)
subset_2298["wksgest"] # length in weeks of all four pregnancies

In [ ]:
subset_2298["wksgest"].mean()

In [ ]:
# birth weight of the first baby born to the respondent with caseid 5013
subset_5013 = preg.query("caseid == 5013 and pregordr == 1")
subset_5013

In [ ]:
subset_5013["totalwgt_kg"]